## Making the imports

In [ ]:
import numpy as np # linear algebra
import torch as torch
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
%pip install seaborn plotly
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff


The dataset we will use is the [Toxic comments dataset](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge). Furthermore we will be dropping a few columns that are not relevant.

In [2]:
train = pd.read_csv('../../Datasets/multilingual_toxic_comment/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('../../Datasets/multilingual_toxic_comment/validation.csv')
test = pd.read_csv('../../Datasets/multilingual_toxic_comment/test.csv')

In [3]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [4]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

We will check the max. number of words present in a comment, which would be halpful for the padding later.

In [5]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

define a function for getting the auc score for validation

In [6]:
def roc_auc(predictions, target):
    # This method returns the AUC score when given the Predictions and Labels

    fpr, tpr, thresholds = metrics.roc_auc(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

# Data preparation

In [7]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values,             stratify=train.toxic.values, test_size=0.2, random_state=42)